# An Introduction to Optimisation via Simulation
## A tutorial for the Simulation Workshop 2020
By Christine S.M Currie and Tom Monks

In [4]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [5]:
from ovs.toy_models import (custom_guassian_model,
                            guassian_sequence_model,
                            random_guassian_model,
                            ManualOptimiser)

This tutorial will be making use of the following models.  

### Creating a simple sequence of normal distributions

A simple model can be created using the `guassian_sequance_model()` function.  This creates a simulation model where the output of each design follows a $N -(\mu_{i}, 1.0)$ :

The function accepts three keyword arguments:

* **start** - int, the first mean in the sequence (inclusive)
* **end** - int, the last mean int the sequence (inclusive)
* **step** - int, the difference between mean i and mean i + 1.

For example, the following code creates a simulation model with 10 designs with means 1 to 10 and unit variance.

```python
model = guassian_sequence_model(1, 10)
```

To create a simulation model with 5 designs where $\mu_{i+1} - \mu_i = 2 $ :

```python
model = guassian_sequence_model(1, 10, step=2)
```

### Creating a custom model with known designs
Instead of a sequence of normal distributions with unit variance, it is possible to create a custom set of designs with varying variances. Use the `custom_guassian_model` function for this task.  For example to create a custom set of designs: 

```python
means = [5, 8, 1, 2, 1, 7]
variances = [0.1, 1.2, 1.4, 0.3, 0.8]

custom_model = custom_guassian_model(means, variances)
```

The following code demonstrates how to create a sequence of 100 designs with variances that are 10% of the mean. 

```python
n_designs = 100
means = [i for i in range(n_designs+1)]
variances = [j*0.1 for j in range(n_designs+1)]

custom_model = custom_guassian_model(means, variances)
```


### Creating a model with randomly sampled designs

To create a model with a set of unknown designs (within a specified mean and variance tolerance) use 

```python
mean_low = 1.0
mean_high = 15.0
var_low = 0.1
var_high = 2.0
n_designs = 15

model = random_guassian_model(mean_low, mean_high, var_low, var_high, n_designs)
```

Where:
* **mean_low** - float, a lower bound on the means of the output distributions
* **mean_high** - float, an upper bound on the means
* **var_low** - float, a lower bound on the variance of the output distributions
* **var_high** - float, an upper bound on the variances.
* **n_designs** - int, the number of designs to create.

### A manual optimisation framework

Before using the **Optimisation via Simulation** procedures, it is recommended that you get a feel for the framework in which the OvS procedures operate.  To do this we will create some models and explore them using a `ManualOptimiser`.  This allows the user to run independent and multiple replications of the model yourself independent of any algorithm.  The `ManualOptimiser` keeps track of the means, variances and number of replications run for each design.

A `ManualOptimiser` object requires two parameters when it is created.  

* model - object, e.g. a model that is a sequence of normal distributions
* n_designs - the number of designs to be considered.

In [6]:
manual_opt = ManualOptimiser(model=guassian_sequence_model(1, 10),
                             n_designs=10)

* We can print the optimiser object to help us remember what parameters we set.

In [7]:
print(manual_opt)

ManualOptimiser(model=BanditCasino(), n_designs=10, verbose=False)


* Follow Law and Kelton's advice and run 3 initial replications of each design.

In [8]:
manual_opt.simulate_designs(replications=3)

The optimiser keeps track of the allocation of replications across each design.  For efficiency it doesn't store each individual observation, but it does compute a running mean and variance.

* Let's have a look at the replication allocation between and results of each design.

In [9]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [10]:
np.set_printoptions(precision=1) # this is a hack to view at 1 decimal place.
manual_opt.means

array([ 0.1,  2.4,  2.6,  3.8,  4.7,  6.3,  7.7,  7.3,  9.6, 10.4])

In [11]:
manual_opt.sigmas

array([0.6, 0.3, 0.2, 0.4, 0.4, 0.6, 0.5, 3.1, 1.3, 0.8])

Now let's run 1 additional replication of the top 3 designs. 
Note - in python arrays are **zero indexed**.  This means that design 1 has index 0.

In [12]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=1)

In [13]:
manual_opt.allocations

array([3, 3, 3, 3, 3, 3, 3, 4, 4, 4], dtype=int32)

In [14]:
manual_opt.simulate_designs(design_indexes=[7, 8, 9], replications=2)

In [15]:
manual_opt.means

array([0.1, 2.4, 2.6, 3.8, 4.7, 6.3, 7.7, 7.2, 9. , 9.6])

In [16]:
manual_opt.sigmas

array([0.6, 0.3, 0.2, 0.4, 0.4, 0.6, 0.5, 1.4, 1.3, 1.8])

### Manual Optimisation of a model with unknown means.

Now have a go yourself.  This time create a model with random designs.  Run as many replications of each design as you think is neccessary to make a decision abouit which design is best.  Here we define best as the design with the largest mean value.

In [17]:
rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

manual_opt = ManualOptimiser(model=rnd_model, n_designs=10)

In [18]:
#insert your code here...


## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [19]:
from ovs.indifference_zone import KN

In [176]:
#something not quite right with KN.  
#quite often is incorrect with random model, but does a lot better with 
#sequence.  This suggests taht something wrong with contenders screening...
rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

kn = KN(model=rnd_model, 
        n_designs=10, 
        delta=0.05, 
        alpha=0.05, 
        n_0=10)

In [177]:
designs = []
for design in rnd_model:
    print(design._mu)
    designs.append(design._mu)

designs = np.array(designs)
np.argmax(designs)

17.268168633114502
12.084057275990038
14.809350533264677
13.558039688693293
19.43056860025211
21.442712544354166
6.785923143202533
16.43235224698076
13.247274435568777
6.601223085249563


5

In [173]:
print(kn)

KN(n_designs=10, delta=0.05, alpha=0.05, n_0=10)


In [178]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[0]
allocations	[151  37  38  39  40  41 148 149 150  36]
total reps	829
means		[17.24 11.95 13.87 13.38 19.49 21.04  6.69 16.52 13.65  6.71]


In [98]:
n_designs = 10
means = [i for i in range(n_designs+1)]
sigmas = [j*0.1 for j in range(n_designs+1)]

guass_model = custom_guassian_model(means, sigmas)

kn = KN(model=guass_model, 
        n_designs=n_designs, 
        delta=0.05, 
        alpha=0.1, 
        n_0=5)

In [99]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[5 5 5 6 6 6 7 7 7 8]
total reps	62
means		[0.  1.  2.  3.  4.1 4.7 5.6 6.9 8.1 9.2]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [100]:
from ovs.fixed_budget import OCBA, OCBAM

In [155]:
n_designs = 10
guass_model = guassian_sequence_model(1, n_designs+1)

rnd_model = random_guassian_model(mean_low=5, mean_high=25, 
                                  var_low=0.5, var_high=2.5,
                                  n_designs=10)

ocba = OCBA(model=rnd_model, 
            n_designs=n_designs, 
            budget=500, 
            delta=5, 
            n_0=5, 
            obj='max')

In [122]:
print(ocba)

OCBA(n_designs=10, budget=500, delta=5, n_0=5, obj=max)


call the `solve()` method to run the optimisation

In [156]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	8
allocations:	[  5   5 293   5   5   5   7   6 164   5]
total reps:	500
means:		[-16.48  -6.51 -23.53 -15.15  -7.11 -15.5  -19.89 -21.54 -23.95 -13.81]
vars:		[0.35 4.42 2.62 5.15 0.07 2.19 6.71 1.58 0.81 9.79]


## Optimal Computing Budget Allocation Top M (OCBA-m)

OCBA-m extended OCBA to identify the top m designs.  

An object of type `OCBAM` takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **m** - int, $m$ the number of top designs to return.  m must be >=2.  For m = 1 see OCBA.
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [118]:
n_designs = 10
guass_model = guassian_sequence_model(1, n_designs+1)

ocbam = OCBAM(model=guass_model, 
              n_designs=n_designs, 
              budget=500, 
              delta=5, 
              n_0=5, 
              m=2,
              obj='max')

In [119]:
print(ocbam)

OCBA(n_designs=10, m=2, budget=500, delta=5, n_0=5, obj=max)


In [120]:
results = ocbam.solve()
print('best designs:\t{}'.format(np.sort(results)))
print('allocations:\t{}'.format(ocbam._allocations))
print('total reps:\t{}'.format(ocbam._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocbam._means))
print('vars:\t\t{0}'.format(ocbam._vars))
print('SEs:\t\t{0}'.format(ocbam._ses))

best designs:	[8 9]
allocations:	[ 10   8   5  19  17  24  35  61 161 160]
total reps:	500
means:		[1.02 2.14 2.16 4.16 5.31 6.3  6.91 8.15 9.   9.93]
vars:		[0.92 0.43 0.09 1.43 0.75 0.93 1.26 0.93 0.81 0.95]
SEs:		[0.3  0.23 0.14 0.27 0.21 0.2  0.19 0.12 0.07 0.08]


# Evaluation of the methods

In [ ]:
from ovs.evaluation import Experiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

In [ ]:
designs = guassian_bandit_sequence(1, 101)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [ ]:
results = exp.execute()

In [ ]:
results.p_correct_selections

In [ ]:
results.selections

In [ ]:
results.correct_selections

## Try different budgets

In [ ]:
from ovs.evaluation import GridExperiment

In [ ]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

param_grid = {'model':[environment],
              'budget':[100, 200, 300, 400, 500], 
              'n_designs':[len(designs)],
              'delta':[1, 5,10],
              'n_0':[1, 5],
              'min':[False]
              }


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=1000)

In [ ]:
results = exp.fit()

In [ ]:
results